# WIP Testing

In [ ]:
# from keras.layers import Dense
# from keras.models import Sequential

import numpy as np
import matplotlib.pyplot as plt


%matplotlib notebook


def load_binary(filename):
    with open(filename, 'rb') as fp:
        values = np.fromfile(fp, dtype='float32')
    return values


# model = Sequential()
# model.add(Dense(2, input_dim=1))
# model.add(Dense(1))
# model.compile(optimizer='adam', loss='mse')

x = load_binary('data/test_001.bin')
# data = np.array(x[1337]).reshape((1, 1))
# p = model.predict(data)

# print(p)

plt.plot(load_binary('data/test_001.bin'))
plt.tight_layout()
plt.show()